In [1]:
import polars as pl
import polars_ds as pds
import numpy as np

# This notebook illustrates the basic usage of this package

You need to create an environment with this package installed to run this notebook. (v0.3.4+)

## New in v0.3.4

You can now access the following queries by calling them directly from pds (e.g. pds.query_lstsq, etc.)

1. All knn related queries
2. All lstsq (linear regression) related queries
3. All graph related queries
4. Most common metrics
5. Miscallenous, which are self-explanatory by the names

More will be added if appropriate. For most occasions, in pds.func() arguments, you can use either str or pl.Expr. For example:

```python
df.select(
    pds.query_lstsq_report(
        pl.col("x1"), "x2", # str | pl.Expr
        target = "y",
        add_bias=False
    ).alias("report")
).unnest("report")

shape: (2, 5)
┌──────────┬───────┬────────────┬────────────┬───────┐
│ feat_idx ┆ coeff ┆ std_err    ┆ t          ┆ p>|t| │
│ ---      ┆ ---   ┆ ---        ┆ ---        ┆ ---   │
│ u16      ┆ f64   ┆ f64        ┆ f64        ┆ f64   │
╞══════════╪═══════╪════════════╪════════════╪═══════╡
│ 0        ┆ 2.0   ┆ 2.3854e-16 ┆ 8.3842e15  ┆ 0.0   │
│ 1        ┆ -1.0  ┆ 9.0158e-17 ┆ -1.1092e16 ┆ 0.0   │
└──────────┴───────┴────────────┴────────────┴───────┘

```

# New in v0.3.5

It is much easier to generate randoms now.

```python
df.with_columns(
    pds.random().alias("[0, 1)"),
    pds.random_normal(pl.col("a").mean(), 1.0).alias("Normal"),
    pds.random_int(0, 10).alias("Int from [0, 10)"),
)
```

# Num Extensions

In [2]:
size = 10_000
df = pl.DataFrame({
    "f": np.sin(list(range(size)))
    , "time_idx": range(size)
    , "dummy": ["a"] * (size // 2) + ["b"] * (size // 2)
    , "a": np.random.random(size = size)
    , "b": np.random.random(size = size)
    , "x1" : range(size)
    , "x2" : range(size, size + size)
    , "y": range(-size, 0)
    , "actual": np.round(np.random.random(size=size)).astype(np.int32)
    , "predicted": np.random.random(size=size)
    , "dummy_groups":["a"] * (size//2) + ["b"] * (size//2) 
})
df.head()

f,time_idx,dummy,a,b,x1,x2,y,actual,predicted,dummy_groups
f64,i64,str,f64,f64,i64,i64,i64,i32,f64,str
0.0,0,"""a""",0.398669,0.243315,0,10000,-10000,0,0.095293,"""a"""
0.841471,1,"""a""",0.614471,0.205741,1,10001,-9999,1,0.289447,"""a"""
0.909297,2,"""a""",0.966153,0.206509,2,10002,-9998,0,0.536976,"""a"""
0.14112,3,"""a""",0.1265,0.423519,3,10003,-9997,1,0.378159,"""a"""
-0.756802,4,"""a""",0.730213,0.820924,4,10004,-9996,1,0.502854,"""a"""


In [3]:
# Column-wise Jaccard Similarity. Result should be 0 as they are distinct
df.select(
    pl.col("x1").num.jaccard(pl.col("x2"))
)

x1
f64
0.0


In [4]:
# FFT. First is real part, second is complex part
# By default, this behaves the same as np's rfft, which returns a non-redundant 
# compact representation of fft output.
df.select(
    pl.col("f").num.rfft()
).head()

f
"array[f64, 2]"
"[1.939505, 0.0]"
"[1.939506, 0.000209]"
"[1.939508, 0.000418]"
"[1.939512, 0.000627]"
"[1.939518, 0.000835]"


In [5]:
# FFT. But return the full length
df.select(
    pl.col("f").num.rfft(return_full=True)
).shape

(10000, 1)

In [6]:
# Convolution (by FFT). 
# Modes: `same`, `left` (left-aligned same), `right` (right-aligned same), `valid` or `full`
# Currently slower than SciPy but provides parallelism because of Polars
df.select(
    pl.col("f").num.convolve([-1, 0, 0, 0, 1], mode = "full"),
    pl.col("a").num.convolve([-1, 0, 0, 0, 1], mode = "full"),
    pl.col("b").num.convolve([-1, 0, 0, 0, 1], mode = "full"),
).head()

f,a,b
f64,f64,f64
1.3944e-15,-0.398669,-0.243315
-0.841471,-0.614471,-0.205741
-0.909297,-0.966153,-0.206509
-0.14112,-0.1265,-0.423519
0.756802,-0.331544,-0.577609


In [7]:
# Least Square (Linear Regression)
df.select(
    pds.query_lstsq(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    )
)

y
list[f64]
"[2.0, -1.0]"


In [8]:
df.select(
    pds.query_lstsq_report(
        # str | pl.Expr
        "x1", "x2",
        target = pl.col("y"),
        add_bias=False
    ).alias("report")
).unnest("report")

feat_idx,coeff,std_err,t,p>|t|
u16,f64,f64,f64,f64
0,2.0,2.3854e-16,8.3842e15,0.0
1,-1.0,9.0158e-17,-1.1092e16,0.0


In [9]:
df.lazy().select(
    pds.query_lstsq(
        pl.col("x1"), pl.col("x2"),
        target = "y", # We can either put pl.col("y") here or just the string "y"
        add_bias=False
    )
).collect()

y
list[f64]
"[2.0, -1.0]"


In [10]:
df.select(
    pds.query_lstsq(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    ).over(pl.col("dummy"))
).head() 

coeffs
list[f64]
"[2.0, -1.0]"
"[2.0, -1.0]"
"[2.0, -1.0]"
"[2.0, -1.0]"
"[2.0, -1.0]"


In [11]:
# If you want prediction and residue instead of coefficients
df.select(
    "x1",
    "x2",
    "y",
    # This is equivalent to pds.query_lstsq, with y being the target. Linters, however, will not pick up this syntax
    # And you have to use pl.col("..") instead of just a str
    (pl.col("y").num.lstsq(pl.col("x1"), pl.col("x2"), add_bias=False, return_pred=True)).alias("prediction")
).unnest("prediction").head()

x1,x2,y,pred,resid
i64,i64,i64,f64,f64
0,10000,-10000,-10000.0,5.8208e-11
1,10001,-9999,-9999.0,5.8208e-11
2,10002,-9998,-9998.0,5.8208e-11
3,10003,-9997,-9997.0,5.8208e-11
4,10004,-9996,-9996.0,5.8208e-11


In [12]:
df.group_by("dummy").agg(
    pl.col("y").num.lstsq(pl.col("x1"), pl.col("x2"), add_bias=False)
)

dummy,coeffs
str,list[f64]
"""b""","[2.0, -1.0]"
"""a""","[2.0, -1.0]"


In [13]:
# Rolling regression, kind of slow rn
df.lazy().rolling(
    index_column = pl.col("time_idx").set_sorted(),
    period = "30i",
    # offset = "-1i"
).agg(
    pds.query_lstsq(pl.col("x1"), pl.col("x2"), target = pl.col("y"), add_bias=False).alias("coefficients")
).slice(offset = 30).select(
    "time_idx",
    "coefficients",
).collect().head(10)

time_idx,coefficients
i64,list[f64]
30,"[2.0, -1.0]"
31,"[2.0, -1.0]"
32,"[2.0, -1.0]"
33,"[2.0, -1.0]"
34,"[2.0, -1.0]"
35,"[2.0, -1.0]"
36,"[2.0, -1.0]"
37,"[2.0, -1.0]"
38,"[2.0, -1.0]"


In [14]:
# Conditional Entropy, should be 0 because x1 is an ID
df.select(
    pl.col("y").num.cond_entropy(pl.col("x1"))
)

y
f64
-0.0


# ML Metrics

In [15]:
df.group_by("dummy_groups").agg(
    pl.col("actual").metric.l2_loss(pl.col("predicted")).alias("l2"),
    pl.col("actual").metric.log_loss(pl.col("predicted")).alias("log loss"),
    # Or equivalently pl.col("actual").metric.binary_metrics_combo(pl.col("predicted"))
    pds.query_binary_metrics(actual="actual", pred="predicted").alias("combo")
).unnest("combo")


dummy_groups,l2,log loss,precision,recall,f,average_precision,roc_auc
str,f64,f64,f64,f64,f64,f64,f64
"""b""",0.324628,0.980689,0.515789,0.527092,0.26069,0.51198,0.518029
"""a""",0.340682,1.029475,0.481124,0.486004,0.241776,0.488724,0.485512


# Str Extension

In [16]:
size = 100_000
df2 = pl.DataFrame({
    "sen":["Hello, world! I'm going to church."] * size,
    "word":["words", "word"] * (size //2)
})
df2.head()

sen,word
str,str
"""Hello, world! …","""words"""
"""Hello, world! …","""word"""
"""Hello, world! …","""words"""
"""Hello, world! …","""word"""
"""Hello, world! …","""words"""


In [17]:
# Tokenize
df2.select(
    pl.col("sen").str.to_lowercase().str2.tokenize().explode().unique()
)

sen
str
"""hello"""
"""to"""
"""church"""
"""world"""
"""going"""


In [18]:
df2.select(
    pl.col("sen").str.to_lowercase().str2.tokenize(stem=True).explode().unique()
)

sen
str
"""church"""
"""hello"""
""""""
"""world"""
"""go"""


In [19]:
df2.select(
    pl.col("word").str2.levenshtein("world")
).head()

word
u32
2
1
2
1
2


In [20]:
# Damerau-Levenshtein
df2.select(
    pl.col("word").str2.d_levenshtein("world")
).head()

word
u32
2
1
2
1
2


In [21]:
df2.select(
    pl.col("word").str2.levenshtein("world", return_sim = True)
).head()

word
f64
0.6
0.8
0.6
0.8
0.6


In [22]:
df2.filter(
    # This is way faster than computing ditance and then doing a filter
    pl.col("word").str2.levenshtein_filter("world", 1) # <= 1. 
).head()

sen,word
str,str
"""Hello, world! …","""word"""
"""Hello, world! …","""word"""
"""Hello, world! …","""word"""
"""Hello, world! …","""word"""
"""Hello, world! …","""word"""


In [23]:
df = pl.DataFrame({
    "word":["apple", "banana", "pineapple", "asasasas", "sasasass"],
    "other_data": [1,2,3,4,5]
})
gibberish = ["asasasa", "sasaaasss", "asdasadadfa"]

In [24]:
df.filter(
    pl.col("word").str2.similar_to_vocab(
        vocab = gibberish,
        threshold = 0.5,
        metric = "lv", # Levenshtein similarity. Other options: dleven, osa, jw
        strategy = "any" # True if the word is similar to any word in vocab. Other options: "all", "avg"
    )
)

word,other_data
str,i64
"""asasasas""",4
"""sasasass""",5


In [25]:
df.select(
    pl.col("word").str2.levenshtein("asasasa", return_sim=True).alias("asasasa"),
    pl.col("word").str2.levenshtein("sasaaasss", return_sim=True).alias("sasaaasss"),
    pl.col("word").str2.levenshtein("asdasadadfa", return_sim=True).alias("asdasadadfa"),
    pl.col("word").str2.fuzz("apples").alias("LCS based Fuzz match - apples"),
    pl.col("word").str2.osa("apples", return_sim = True).alias("Optimal String Alignment - apples"),
    pl.col("word").str2.jw("apples").alias("Jaro-Winkler - apples"),
)


asasasa,sasaaasss,asdasadadfa,LCS based Fuzz match - apples,Optimal String Alignment - apples,Jaro-Winkler - apples
f64,f64,f64,f64,f64,f64
0.142857,0.111111,0.090909,0.833333,0.833333,0.966667
0.428571,0.333333,0.272727,0.166667,0.0,0.444444
0.111111,0.111111,0.090909,0.555556,0.444444,0.5
0.875,0.666667,0.545455,0.25,0.25,0.527778
0.75,0.777778,0.454545,0.25,0.25,0.527778


# Stats Extension

In [26]:
import numpy as np

df = pl.DataFrame({
    "a": [None, None] + list(np.random.normal(size = 998))
})
df.head()

a
f64
null
null
1.240444
0.340659
0.350871


In [27]:
# Genenrate random numbers, respecting null positions in reference column (pl.col("a"))
df.with_columns(
    pl.col("a").stats.rand_normal(mean = 0.5, std = 1., respect_null=True).alias("random")
).head()

a,random
f64,f64
null,null
null,null
1.240444,0.443088
0.340659,1.913403
0.350871,0.427976


In [28]:
# Genenrate random string
df.with_columns(
    pl.col("a").stats.rand_str(min_size = 1, max_size = 5, respect_null=True).alias("random_str")
).head()

a,random_str
f64,str
null,null
null,null
1.240444,"""jQB"""
0.340659,"""J"""
0.350871,"""rD7I"""


In [29]:
# Genenrate fixed size random string
df.with_columns(
    pl.col("a").stats.rand_str(min_size = 5, max_size = 5, respect_null=True).alias("random_str")
).head()

a,random_str
f64,str
null,null
null,null
1.240444,"""MONWH"""
0.340659,"""LS2f8"""
0.350871,"""vSSrv"""


In [30]:
df.with_columns(
    # Sample from normal distribution, using reference column "a" 's mean and std
    pl.col("a").stats.rand_normal().alias("test1") 
    # Sample from uniform distribution, with low = 0 and high = "a"'s max, and respect the nulls in "a"
    , pl.col("a").stats.rand_uniform(low = 0., high = None, respect_null=True).alias("test2")
).with_columns(
    # Add a random pertubation to test1
    pl.col("test1").stats.perturb(epsilon=0.001).alias("test1_perturbed")
).head()

a,test1,test2,test1_perturbed
f64,f64,f64,f64
null,0.985711,null,0.985275
null,0.318913,null,0.318985
1.240444,0.205293,2.983992,0.204986
0.340659,-0.501305,1.816323,-0.50091
0.350871,0.768977,0.012952,0.76895


In [31]:
# New in v0.3.5
# This way, we don't have a reference column, so we cannot respect nulls, but is more convenient to use.
df.with_columns(
    pds.random().alias("[0, 1)"),
    pds.random_normal(pl.col("a").mean(), pl.col("a").std()).alias("Normal"),
    pds.random_int(0, 10).alias("Int from [0, 10)"),
).head()

a,"[0, 1)",Normal,"Int from [0, 10)"
f64,f64,f64,i32
null,0.786725,1.073521,7
null,0.886659,2.184259,9
1.240444,0.444681,-1.981882,1
0.340659,0.043816,1.009044,8
0.350871,0.727963,-0.880942,5


In [32]:
# Genenrate 2 random sample, both normally distributed
# Run Welch's t test on them, p value should be big since they have equal mean
# Run a normality test. Again, p value should be big since they are normally distributed 

df.with_columns(
    pl.col("a").stats.rand_normal(mean = 0.5, std = 1.).alias("test1")
    , pl.col("a").stats.rand_normal(mean = 0.5, std = 2.).alias("test2")
).select(
    pl.col("test1").stats.ttest_ind(pl.col("test2"), equal_var = False).alias("t-test")
    , pl.col("test1").stats.normal_test().alias("normality_test")
).select(
    pl.col("t-test").struct.field("statistic").alias("t-tests: statistics")
    , pl.col("t-test").struct.field("pvalue").alias("t-tests: pvalue")
    , pl.col("normality_test").struct.field("statistic").alias("normality_test: statistics")
    , pl.col("normality_test").struct.field("pvalue").alias("normality_test: pvalue")
)

t-tests: statistics,t-tests: pvalue,normality_test: statistics,normality_test: pvalue
f64,f64,f64,f64
-2.128707,0.033441,0.426249,0.808055


In [33]:
size = 5_000
df = pl.DataFrame({
    "market_id": range(size),
    "group1": np.random.random(size=size),
    "group2": np.random.random(size=size),
    "category_1": np.random.randint(low=0, high=5, size=size),
    "category_2":np.random.randint(low=0, high=10, size=size)
}).with_columns(
    pl.col("market_id").mod(3)
)
df.head(5)

market_id,group1,group2,category_1,category_2
i64,f64,f64,i64,i64
0,0.632681,0.817873,4,9
1,0.01587,0.174498,0,7
2,0.188836,0.931489,2,2
0,0.104825,0.01259,0,9
1,0.149639,0.252975,0,4


In [34]:
# In dataframe statistical tests!
df.select(
    pl.col("group1").stats.ttest_ind(pl.col("group2"), equal_var = True).alias("t-test"),
    pl.col("category_1").stats.chi2(pl.col("category_2")).alias("chi2-test"),
    pl.col("category_1").stats.f_test(pl.col("group1")).alias("f-test")
)

t-test,chi2-test,f-test
struct[2],struct[2],struct[2]
"{-1.093978,0.273991}","{34.328848,0.548196}","{0.387345,0.817847}"


In [35]:
# Can also be done in group by context
df.group_by("market_id").agg(
    pl.col("group1").stats.ttest_ind(pl.col("group2"), equal_var = False).alias("t-test"),
    pl.col("category_1").stats.chi2(pl.col("category_2")).alias("chi2-test"),
    pl.col("category_1").stats.f_test(pl.col("group1")).alias("f-test")
)

market_id,t-test,chi2-test,f-test
i64,struct[2],struct[2],struct[2]
0,"{0.354686,0.722847}","{39.348289,0.322374}","{0.671904,0.61148}"
1,"{-1.297396,0.194585}","{42.198747,0.220659}","{0.536813,0.708714}"
2,"{-0.933237,0.350766}","{25.792637,0.896074}","{0.333421,0.855589}"


# Nearest Neighbors Related Tasks

These queries can be very slow when data/dimension gets huge, even when processed in parallel.

In [36]:
import polars_ds as pds
size = 2000
df = pl.DataFrame({
    "id": range(size), 
    "val1": np.random.random(size=size), 
    "val2": np.random.random(size=size), 
    "val3": np.random.random(size=size),
    "r": np.random.random(size=size),
    "rh": np.random.random(size=size)*10,
}).with_columns(
    pl.col("id").cast(pl.UInt32)
)

In [37]:
# Get neighbor count. The point itself is always considered a neighbor to itself.
df.with_columns(
    pds.query_nb_cnt(
        0.1, # radius 
        pl.col("val1"), "val2", "val3", # Columns used as the coordinates in n-d space, str | pl.Expr 
        dist = "inf", # L Infinity distance 
        parallel = True 
    ).alias("nb_l_inf_cnt")
).head() 

id,val1,val2,val3,r,rh,nb_l_inf_cnt
u32,f64,f64,f64,f64,f64,u32
0,0.506973,0.494173,0.892488,0.250596,3.805498,17
1,0.464808,0.770314,0.545069,0.618297,4.754389,21
2,0.07338,0.908391,0.712785,0.597162,6.688595,13
3,0.689477,0.224162,0.224163,0.872439,4.588719,10
4,0.20452,0.093144,0.342247,0.940545,7.585782,20


In [38]:
df.with_columns(
    pds.query_nb_cnt(
        pl.col("r"), # radius be an expression too
        "val1", "val2", "val3", # Columns used as the coordinates in n-d space, str | pl.Expr 
        dist = "l1", # L 1 distance 
        parallel = True 
    ).alias("nb_l1_r_cnt")
).head()

id,val1,val2,val3,r,rh,nb_l1_r_cnt
u32,f64,f64,f64,f64,f64,u32
0,0.506973,0.494173,0.892488,0.250596,3.805498,42
1,0.464808,0.770314,0.545069,0.618297,4.754389,537
2,0.07338,0.908391,0.712785,0.597162,6.688595,216
3,0.689477,0.224162,0.224163,0.872439,4.588719,894
4,0.20452,0.093144,0.342247,0.940545,7.585782,845


In [39]:
# Get ids of the k nearest neighbors. 
# The point itself is always considered a neighbor to itself, so k + 1 elements will be returned.
df.with_columns(
    pds.query_knn_ptwise(
        pl.col("val1"), pl.col("val2"), pl.col("val3"), # Columns used as the coordinates in n-d space
        index = "id",  # pl.col("id"), str | pl.Expr
        k = 3, 
        dist = "l2", # squared l2
        parallel = True
    ).alias("best friends")
).head() 

id,val1,val2,val3,r,rh,best friends
u32,f64,f64,f64,f64,f64,list[u32]
0,0.506973,0.494173,0.892488,0.250596,3.805498,"[0, 597, … 549]"
1,0.464808,0.770314,0.545069,0.618297,4.754389,"[1, 705, … 724]"
2,0.07338,0.908391,0.712785,0.597162,6.688595,"[2, 1186, … 1553]"
3,0.689477,0.224162,0.224163,0.872439,4.588719,"[3, 103, … 220]"
4,0.20452,0.093144,0.342247,0.940545,7.585782,"[4, 1907, … 531]"


In [40]:
# Get all neighbors within radius r
# The point itself is always considered a neighbor to itself.
print(df.select(
    pl.col("id"),
    pds.query_radius_ptwise(
        pl.col("val1"), pl.col("val2"), pl.col("val3"), # Columns used as the coordinates in n-d space
        index = pl.col("id"),
        r = 0.1, 
        dist = "l2", # actually this is squared l2
        parallel = True
    ).alias("best friends"),
).with_columns( # -1 to remove the point itself
    (pl.col("best friends").list.len() - 1).alias("best friends count")
).head())

shape: (5, 3)
┌─────┬──────────────────┬────────────────────┐
│ id  ┆ best friends     ┆ best friends count │
│ --- ┆ ---              ┆ ---                │
│ u32 ┆ list[u32]        ┆ u32                │
╞═════╪══════════════════╪════════════════════╡
│ 0   ┆ [0, 597, … 713]  ┆ 221                │
│ 1   ┆ [1, 705, … 1004] ┆ 264                │
│ 2   ┆ [2, 1186, … 613] ┆ 117                │
│ 3   ┆ [3, 103, … 1409] ┆ 220                │
│ 4   ┆ [4, 1907, … 992] ┆ 186                │
└─────┴──────────────────┴────────────────────┘


In [41]:
# Get ids of the k nearest neighbors and distances
# The point itself is always considered a neighbor to itself, so k + 1 elements will be returned.
df.with_columns(
    pds.query_knn_ptwise(
        pl.col("val1"), pl.col("val2"), pl.col("val3"), # Columns used as the coordinates in n-d space
        index = pl.col("id"),
        k = 3, 
        dist = "l2", # actually this is squared l2
        parallel = True,
        return_dist = True
    ).alias("best_friends_w_dist")
).unnest("best_friends_w_dist").head()

id,val1,val2,val3,r,rh,idx,dist
u32,f64,f64,f64,f64,f64,list[u32],list[f64]
0,0.506973,0.494173,0.892488,0.250596,3.805498,"[0, 597, … 549]","[0.0, 0.002902, … 0.005174]"
1,0.464808,0.770314,0.545069,0.618297,4.754389,"[1, 705, … 724]","[0.0, 0.001036, … 0.002345]"
2,0.07338,0.908391,0.712785,0.597162,6.688595,"[2, 1186, … 1553]","[0.0, 0.00398, … 0.008144]"
3,0.689477,0.224162,0.224163,0.872439,4.588719,"[3, 103, … 220]","[0.0, 0.001308, … 0.008485]"
4,0.20452,0.093144,0.342247,0.940545,7.585782,"[4, 1907, … 531]","[0.0, 0.001585, … 0.003495]"


In [42]:
# Filter to only points near the given point
df.filter(
    pds.query_radius_at_pt(
        pl.col("val1"), pl.col("val2"), pl.col("val3"), # Columns used as the coordinates in n-d space
        pt = [0.5, 0.5, 0.5],
        r = 0.2,
        dist = "l2" # actually this is squared l2, so this is asking for squared l2 <= 0.2
    )
).head()

id,val1,val2,val3,r,rh
u32,f64,f64,f64,f64,f64
0,0.506973,0.494173,0.892488,0.250596,3.805498
1,0.464808,0.770314,0.545069,0.618297,4.754389
3,0.689477,0.224162,0.224163,0.872439,4.588719
5,0.391367,0.445461,0.586592,0.686767,8.083743
6,0.373096,0.427133,0.512025,0.915182,8.652561


In [43]:
# Haversine distance is available when dimension is 2
df.filter(
    pds.query_radius_at_pt(
        pl.col("val1"), pl.col("val2"), # Columns used as the coordinates in n-d space
        pt = [0.5, 0.5],
        r = 10, # in km
        dist = "h" 
    )
).head()

id,val1,val2,val3,r,rh
u32,f64,f64,f64,f64,f64
0,0.506973,0.494173,0.892488,0.250596,3.805498
27,0.449354,0.464936,0.386711,0.934358,5.930981
108,0.453358,0.455474,0.254709,0.369354,6.854675
110,0.578392,0.491167,0.513575,0.869283,4.351892
123,0.533402,0.425421,0.084883,0.937096,3.958402


In [44]:
df.filter(
    pds.query_radius_at_pt(
        pl.col("val1"), pl.col("val2"), 
        pt = [0.5, 0.5],
        # radius can also be an existing column in the dataframe.
        r = pl.col("rh"), 
        dist = "h" 
    )
).head()

id,val1,val2,val3,r,rh
u32,f64,f64,f64,f64,f64
0,0.506973,0.494173,0.892488,0.250596,3.805498
160,0.505177,0.527278,0.191167,0.087277,6.792438
386,0.502902,0.525816,0.254772,0.33871,8.917753
596,0.49003,0.453594,0.355865,0.149327,9.891922
597,0.48431,0.499775,0.941036,0.381983,4.015817


In [45]:
friends = df.select(
    pl.col("id").cast(pl.UInt64),
    pds.query_radius_ptwise(
        # Columns used as the coordinates in n-d space
        pl.col("val1"), pl.col("val2"), 
        index=pl.col("id"),
        r = 0.02, 
        dist = "l2",
    ).alias("friends")
).with_columns(
    pl.col("friends").list.len().alias("count")
)
friends.head()

id,friends,count
u64,list[u32],u32
0,"[0, 1667, … 1999]",126
1,"[1, 1393, … 510]",147
2,"[2, 1415, … 1315]",87
3,"[3, 799, … 1363]",127
4,"[4, 632, … 693]",108


# Simple Graph Queries

There is limited functionality in the Graph module currently. E.g. Only constant cost per edge.

Graph queries are very expensive.

In [46]:
# friends.select(
#     pl.col("friends").graph.eigen_centrality() # .arg_max()
# ).head()

In [47]:
# Turn friends to a table suitable for graph analytics
df_graph = friends.select(
    pl.col("id"),
    pl.col("friends"),
).explode(pl.col("friends")).with_columns(
    pl.col("id").cast(pl.UInt32),
    pl.col("friends").cast(pl.UInt32),
)
df_graph.head()

id,friends
u32,u32
0,0
0,1667
0,733
0,1339
0,1586


In [48]:
df_graph.select(
    # Shortest path to the node with id = 3
    # Node and link can be str | pl.Expr
    pds.query_shortest_path(node = "id", link = pl.col("friends"), target = 3, cost = None, parallel=True).alias("shortest_path")
).unnest("shortest_path").sort("id")

id,path
u32,list[u32]
0,"[1632, 621, 3]"
1,"[606, 1730, … 3]"
2,"[1419, 1144, … 3]"
3,[]
4,"[1199, 124, … 3]"
…,…
1995,"[996, 1366, 3]"
1996,"[1436, 1087, … 3]"
1997,"[1038, 259, … 3]"


In [49]:
df_graph.select(
    # Almost every node can reach node 3, and the number is the number steps to reach it
    # This is a way faster way to filter results if you don't need the actual path
    pl.col("id").graph.reachable(link="friends", target = 3).alias("reach")
).unnest("reach")

id,reachable,steps
u32,bool,u32
1958,true,4
1270,true,4
157,true,4
1283,true,2
543,true,6
…,…,…
1493,true,7
1705,true,4
926,true,6


In [50]:
relationships = pl.DataFrame({
    "id": range(5),
    "connections":[[1,2,3,4], [2,3], [4], [0,1,2], [1]],
    # Small values means closer
    "close-ness":[[0.4, 0.3, 0.2, 0.1], [0.1, 1.0], [0.5], [0.1, 0.1, 0.1], [0.1]]
}).with_columns(
    pl.col("id").cast(pl.UInt32),
    pl.col("connections").list.eval(pl.element().cast(pl.UInt32))
).explode(
    pl.col("connections"), pl.col("close-ness")
)

relationships.head(50)

id,connections,close-ness
u32,u32,f64
0,1,0.4
0,2,0.3
0,3,0.2
0,4,0.1
1,2,0.1
…,…,…
2,4,0.5
3,0,0.1
3,1,0.1


In [51]:
# To go to node at id = 1, node 0 would rather go to 4 first and then 1.
relationships.select(
    pl.col("id").graph.shortest_path(
        link = "connections",
        target = 1,
        cost = pl.col("close-ness"),
        parallel = False
    ).alias("path")
).unnest("path").head()

id,path,cost
u32,list[u32],f64
2,"[4, 1]",0.6
0,"[4, 1]",0.2
4,[1],0.1
3,[1],0.1
1,[],0.0


In [52]:
# In and out deg
relationships.select(
    pl.col("id").graph.in_out_deg(link=pl.col("connections")).alias("deg")
).unnest("deg")

node,in,out
u32,u32,u32
2,3,1
0,1,4
4,2,1
3,2,3
1,3,2


# String Nearest Neighbors

This might be very slow for very large vocab / column.

In [53]:
df = pl.DataFrame({
    "a":["AAAAA", "ABCABC", "AAAADDD", "ADSDSDS", "WORD"],
    "b":["AAAAT", "ABCACD", "ADSSD", "APPLES", "WORLD"] 
})

In [54]:
# Use Levenshtein to find the nearest neighbor in vocab to word in column a
df.select(
    pl.col("a").str2.similar_words(
        vocab = pl.col("b"),
        k = 1,
        metric = "lv"
    ).alias("similar_words_from_vocab"),
)

similar_words_from_vocab
str
"""AAAAT"""
"""ABCACD"""
"""AAAAT"""
"""ADSSD"""
"""WORLD"""


In [55]:
# Use Levenshtein to find 2 nearest neighbors
df.select(
    pl.col("a").str2.similar_words(
        vocab = pl.col("b"),
        k = 2,
        metric = "lv"
    ).alias("similar_words_from_vocab"),
)

similar_words_from_vocab
list[str]
"[""AAAAT"", ""ADSSD""]"
"[""ABCACD"", ""AAAAT""]"
"[""AAAAT"", ""ABCACD""]"
"[""ADSSD"", ""APPLES""]"
"[""WORLD"", ""ADSSD""]"


In [56]:
# Currently only Levenshtein and hamming are implemented for this
# Empty means nothing in vocab can be compared in the hamming sense with the corresponding word in a
df.select(
    pl.col("a").str2.similar_words(
        vocab = pl.col("b"),
        k = 2,
        threshold = 4, # <= threshold hamming distance away
        metric = "hamming"
    ).alias("similar_words_from_vocab"),
)

similar_words_from_vocab
list[str]
"[""AAAAT"", ""ADSSD""]"
"[""ABCACD""]"
[]
[]
[]


In [57]:
# You may provide a vocab like this
df.select(
    pl.col("a"),
    pl.col("a").str2.similar_words(
        vocab = ["WORLD", "AAAAA", "ABCDEFG", "ZIV", "TQQQ"],
        k = 3,
        metric = "lv"
    ).alias("similar_words_from_vocab"),
)

a,similar_words_from_vocab
str,list[str]
"""AAAAA""","[""AAAAA"", ""ZIV"", ""WORLD""]"
"""ABCABC""","[""ABCDEFG"", ""AAAAA"", ""ZIV""]"
"""AAAADDD""","[""AAAAA"", ""WORLD"", ""ABCDEFG""]"
"""ADSDSDS""","[""ABCDEFG"", ""WORLD"", ""AAAAA""]"
"""WORD""","[""WORLD"", ""ZIV"", ""TQQQ""]"
